In [1]:
from splinter import Browser
import requests
import sys
import time
import getpass
from collections import defaultdict

In [2]:
class QuantumCircuit:
	"""
	This is a class that interfaces with the Alibaba quantum computer.

	Attributes:
		circuit_name (string) : The name you would like to call your circuit. Defaults to a random string. (TODO)
		cookies (dict) : A dictionary of cookies that you get from logging in to alibaba quantumcomputer.ac.cn.
	"""
	def __init__(self, circuit_name, cookies):
		self.cookies = cookies
		self.circuit_name = circuit_name

		self.data = []

		self.circuit_id = None

		self.is_run = False

		self.results = {}

		# Tracks the furthest occupied x value in the rail
		self.rails = defaultdict(int)

	def update_cookies(self, new_cookies):
		"""
		The function to update cookies. Cookies must be updates every once in a while as they expire.

		Parameters:
			new_cookies (dict) : A dictionary of cookies that you get from logging in to alibaba quantumcomputer.ac.cn.
		"""
		self.cookies = new_cookies

	def get_results(self):
		"""
		The function that fills and returns in the results for the most recent run of this quantum circuit.

		Returns:
			If successful, returns the results as a dict.
			If unsuccessful, returns None.
		"""
		url = 'http://quantumcomputer.ac.cn/experiment/resultlist'
		data = {
			"experimentId": self.circuit_id,
			"version":'',
			"_input_charset": "utf-8"
		}

		get_exp = requests.get(url, params=data, cookies=self.cookies)

		res = get_exp.json()

		if "success" in res and res["success"] == True:
			self.results = res
			# print(get_exp.text)
			return self.results
		return None

	def get_csrf(self, url):
		"""
		Helps us sidestep security precautions to avoid cross-site request forgery (csrf).
		"""
		url = url

		r = requests.get(url, cookies=self.cookies)
		temp = r.text
		csrf = temp.split("var csrf = \'")[1].split("\'")[0]

		return csrf

	def new_circuit(self, realOrSim="SIMULATE", bitWidth=10):
		"""
		The function to deploy this circuit as a new circuit on the quantum computer.

		Parameters:
			realOrSim (boolean?) : If you want to run it on a simulation, then keep it as "SIMULATE".
			self.bitWidth (int) : An integer in the range [10, 25] for "SIMULATE" and [11, 11] for a real
				quantum computer that determines the number of qubits you are allocated.

		Returns:
			True if the new circuit was added and False if not.
			Modifies self.circuit_id if successful so that we can edit and run this circuit by referring to
				it through circuit_id.
		"""
		url = 'http://quantumcomputer.ac.cn/experiment/infosave?_input_charset=utf-8'

		data = {
			'name': self.circuit_name,
			'type': realOrSim,
			'bitWidth': bitWidth
		}

		csrf = self.get_csrf('http://quantumcomputer.ac.cn/list.html')
		headers = {
			'X-CSRF-TOKEN': csrf,
			"Referer": "http://quantumcomputer.ac.cn/list.html",
			"Host": "quantumcomputer.ac.cn",
			"Origin": "http://quantumcomputer.ac.cn",
			"Content-Type": "application/json"
		}

		make_new_circuit = requests.post(url, headers=headers, json=data, cookies=self.cookies)

		new_circuit_json = make_new_circuit.json()
		if "success" in new_circuit_json and new_circuit_json["success"] == True:
			self.circuit_id = str(new_circuit_json["data"])
			return True
		return False

	def add_gate_single(self, text, x, y, gateDetail={}):
		"""
		Adds a custom single qubit gate. Used as a helper function, not directly used by the user.

		Parameters:
			text: The identity of the gate, for example 'H', 'X', 'Y', etc.
			x: The x value of the gate. Gates are run from lowest x value to highest on the same rail.
			y: The value of the rail.
			gateDetail: {}

		Returns:
			Modifies self.data if successful. Later self.data is submitted to the server.
		"""
		edit = {
			"text":text,
			"gateDetail":gateDetail,
			"x":x,
			"y":y
		}

		self.data.append(edit)

	def add_gate_double(self, text, x, y, y1, gateDetail={}):
		"""
		See add_gate_single. Text must be 'CP' for the simulator.
		"""
		edit = {
			"text":text,
			"gateDetail":gateDetail,
			"x":x,
			"y":y,
			"x1":x,
			"y1":y1
		}

		self.data.append(edit)

	def add_gate_triple(self, text, x, y, y1, y2, gateDetail={}):
		"""
		See add_gate_single. Text must be 'CCP' for the simulator.
		"""
		edit = {
			"text":text,
			"gateDetail":gateDetail,
			"x":x,
			"y":y,
			"x1":x,
			"y1":y1,
			"x2":x,
			"y2":y2
		}

		self.data.append(edit)

	def add_H(self, target):
		"""
		Adds a hadamard gate to the target rail.

		Parameters:
			target: The index of the rail that will recieve the gate.
		"""
		last_filled_spot = self.rails[target]
		self.add_gate_single("H", last_filled_spot + 1, target)
		self.rails[target] = last_filled_spot + 1

	def add_X(self, target):
		"""
		Adds a Pauli-X gate to the target rail.

		Parameters:
			target: The index of the rail that will recieve the gate.
		"""
		last_filled_spot = self.rails[target]
		self.add_gate_single("X", last_filled_spot + 1, target)
		self.rails[target] = last_filled_spot + 1

	def add_Y(self, target):
		"""
		Adds a Pauli-Y gate to the target rail.

		Parameters:
			target: The index of the rail that will recieve the gate.
		"""
		last_filled_spot = self.rails[target]
		self.add_gate_single("Y", last_filled_spot + 1, target)
		self.rails[target] = last_filled_spot + 1

	def add_Z(self, target):
		"""
		Adds a Pauli-Z gate to the target rail.

		Parameters:
			target: The index of the rail that will recieve the gate.
		"""
		last_filled_spot = self.rails[target]
		self.add_gate_single("Y", last_filled_spot + 1, target)
		self.rails[target] = last_filled_spot + 1

	def add_M(self, target):
		"""
		Adds a Measurement gate to the target rail.

		Parameters:
			target: The index of the rail that will recieve the gate.
		"""
		last_filled_spot = self.rails[target]
		self.add_gate_single("M", last_filled_spot + 1, target)
		self.rails[target] = last_filled_spot + 1

	def add_CNOT(self, control, target):
		"""
		Adds a controlled-not gate with the control on the control rail and the target on the target rail.

		Parameters:
			control: The index of the control rail.
			target: The index of the target rail.
		"""
		self.add_H(target)
		self.add_X(control)

		last_filled_spot = max(self.rails[control], self.rails[target]) 

		self.add_gate_double("CP", last_filled_spot + 1, control, target)

		self.rails[control] = last_filled_spot + 1
		self.rails[target] = last_filled_spot + 1

		self.add_H(target)
		self.add_X(control)
        
	def add_CCNOT(self, control1, control2, target):
		"""
		Adds a controlled-controlled-not gate.

		Parameters:
			control1: The index of the first control rail.
			control2: The index of the second control rail.
			target: The index of the target rail.
		"""
		self.add_H(target)
		self.add_X(control1)
		self.add_X(control2)

		last_filled_spot = max(self.rails[control1], self.rails[control2], self.rails[target])

		self.add_gate_triple("CCP", last_filled_spot + 1, control1, control2, target)

		self.rails[control1] = last_filled_spot + 1
		self.rails[control2] = last_filled_spot + 1
		self.rails[target] = last_filled_spot + 1

		self.add_H(target)
		self.add_X(control1)
		self.add_X(control2)

	def add_RZ(self, target, angle):
		"""
		Adds an RZ gate to the target rail.

		Parameters:
			target: The index of the rail that will recieve the gate.
			angle: The angle (degrees) of the rotation. Float or int.
		"""
		text = "RZ_" + str(angle)

		last_filled_spot = self.rails[target]
		self.add_gate_single(text, last_filled_spot + 1, target)
		self.rails[target] = last_filled_spot + 1

	def add_RX(self, target, angle):
		"""
		Adds an RX gate to the target rail.

		Parameters:
			target: The index of the rail that will recieve the gate.
			angle: The angle (degrees) of the rotation. Float or int.
		"""
		text = "RX_" + str(angle)

		last_filled_spot = self.rails[target]
		self.add_gate_single(text, last_filled_spot + 1, target)
		self.rails[target] = last_filled_spot + 1

	def push_edits(self):
		"""
		Pushes edits to the alibaba quantum computer.

		Returns:
			True if edits were pushed successfully. False if otherwise.
		"""
		url = 'http://quantumcomputer.ac.cn/experiment/codesave?_input_charset=utf-8'

		data_json = {}

		data_json['experimentId'] = self.circuit_id
		data_json['data'] = self.data
		data_json['code'] = ""

		csrf_url = 'http://quantumcomputer.ac.cn/home.html?id=' + str(self.circuit_id)
		csrf = self.get_csrf(url=csrf_url)
		headers = {
			'X-CSRF-TOKEN': csrf
		}

		edit_circuit = requests.post(url, headers=headers, json=data_json, cookies=self.cookies)

		return edit_circuit.json()["success"]

	def run_circuit(self, shots=100, seed=420, Type="SIMULATE"):
		"""
		Tells the alibaba server to run the circuit. Circuit output data must be later retrieved.

		Returns:
			True if the circuit was successfully run. False if otherwise.
		"""
		url = 'http://quantumcomputer.ac.cn/experiment/submit'
		data = {
			"experimentId": self.circuit_id,
			"type": Type,
			"bitWidth": "",
			"shots": shots,
			"seed": seed,
			"_input_charset": "utf-8"
		}

		csrf_url = 'http://quantumcomputer.ac.cn/home.html?id=' + str(self.circuit_id)
		csrf = self.get_csrf(url=csrf_url)
		headers = {
			'X-CSRF-TOKEN': csrf
		}

		run_circuit = requests.post(url, headers=headers, data=data, cookies=self.cookies)

		if "success" in run_circuit.json():
			self.is_run = True
			return run_circuit.json()["success"]
        
	def run_circuit_get_results(self, shots=100, seed=420, Type='SIMULATE'):
		"""
		Pushes edits to the server, then runs the circuit, and finally retrieves results.

		Returns:
			A dictionary of results if successful. None if not successful.
		"""
		# Pushing the edits to the server
		if not self.push_edits():
			return None

		# Telling the server to run the circuit
		if not self.run_circuit(shots=shots, seed=seed, Type='SIMULATE'):
			return None

		time.sleep(3)

		# Pulling the results from the circuit.
		results = self.get_results()
		return results

Run the code below to log in and generate new cookies.

In [3]:
username = str(input("Username: "))
print('Password: ', end='')
password = getpass.getpass()

url = 'http://quantumcomputer.ac.cn/login'

browser = Browser('firefox')
browser.visit(url)

browser.find_by_xpath('//*[@id="username"]').first.type(username)
browser.find_by_xpath('//*[@id="password"]').first.type(password)
browser.find_by_xpath('//*[@id="qasm-wrapper"]/div[3]/div[2]/div[2]/form/button').first.click()

cookies = browser.cookies.all()

time.sleep(3)

browser.quit()


Username: stevecarrell63
Password: ········


Generate a Bell State:
![hyperlink](https://i.ibb.co/jfDJ04Z/Capture2.png)

In [11]:
# Making a new quantum circuit
qc = QuantumCircuit('Bell State', cookies)

if qc.new_circuit():
    qc.add_H(1)
    qc.add_CNOT(1, 2)
    qc.add_M(1)
    qc.add_M(2)
else:
    print("Failed to make a circuit.")
    
# Run the circuit.
results = qc.run_circuit_get_results(shots=1024, seed=12345)
results['data']['simulateResult'][0]

{'startTime': '2019-05-08 05:26:58',
 'finishTime': '2019-05-08 05:26:59',
 'process': None,
 'data': {'00': '0.50293', '11': '0.49707'},
 'id': 11032,
 'seed': 12345,
 'shots': 1024,
 'measureQubits': [1, 0]}

Implementing a SWAP gate:
![hyperlink](https://i.ibb.co/HDs96Kj/Capture1.png)

In [12]:
class QuantumCircuitWithSwap(QuantumCircuit):
    def add_swap(self, qubit1, qubit2):
        self.add_CNOT(qubit1, qubit2)
        self.add_CNOT(qubit2, qubit1)
        self.add_CNOT(qubit1, qubit2)

qc_swap = QuantumCircuitWithSwap('Using SWAP', cookies)

if qc_swap.new_circuit():
    qc_swap.add_X(1)
    qc_swap.add_swap(1, 2)
    qc_swap.add_M(1)
    qc_swap.add_M(2)
    
# Run the circuit.
results = qc_swap.run_circuit_get_results(shots=1024, seed=12345)
results['data']['simulateResult'][0]

{'startTime': '2019-05-07 04:59:32',
 'finishTime': '2019-05-07 04:59:33',
 'process': None,
 'data': {'10': '1.0'},
 'id': 10987,
 'seed': 12345,
 'shots': 1024,
 'measureQubits': [1, 0]}

Manually implementing $S$, $T$, $T^{\dagger}$, and $\text{Toffoli}$ using RZ:

In [4]:
class QuantumCircuitToffoli(QuantumCircuit):
    def add_T(self, target):
        self.add_RZ(target, 45)
        
    def add_T_dag(self, target):
        self.add_RZ(target, 315)

    def add_S(self, target):
        self.add_RZ(target, 90)
        
    def add_TOFF(self, control1, control2, target):
        self.add_H(target)
        self.add_CNOT(control2, target)
        self.add_T_dag(target)
        self.add_CNOT(control1, target)
        self.add_T(target)
        self.add_CNOT(control2, target)
        self.add_T_dag(target)
        self.add_CNOT(control1, target)
        self.add_T_dag(control2)
        self.add_T(target)
        self.add_H(target)
        self.add_CNOT(control1, control2)
        self.add_T_dag(control2)
        self.add_CNOT(control1, control2)
        self.add_T(control1)
        self.add_S(control2)

Using the $\text{Toffoli}$:
![hyperlink](https://i.ibb.co/NVFGSFW/Capture.png)

In [14]:
qc_toff = QuantumCircuitToffoli('Toffoli', cookies)

if qc_toff.new_circuit():
    qc_toff.add_X(1)
    qc_toff.add_X(2)
    qc_toff.add_TOFF(1, 2, 3)
    qc_toff.add_M(1)
    qc_toff.add_M(2)
    qc_toff.add_M(3)
    
# Run the circuit.
results = qc_toff.run_circuit_get_results(shots=1024, seed=12345)
results['data']['simulateResult'][0]

{'startTime': '2019-05-07 05:00:03',
 'finishTime': '2019-05-07 05:00:04',
 'process': None,
 'data': {'111': '1.0'},
 'id': 10988,
 'seed': 12345,
 'shots': 1024,
 'measureQubits': [2, 1, 0]}

Here is what part of this Toffoli gate decomposition looks like on the CAS-Alibaba Quantum Computer GUI:
![hyperlink](https://i.ibb.co/hsL6Thv/Capture3.png)

Compare this to our built-in CCNOT gate:

In [15]:
qc = QuantumCircuit('CCNOT', cookies)

if qc.new_circuit():
    qc.add_X(1)
    qc.add_X(2)
    qc.add_CCNOT(1, 2, 3)
    qc.add_M(1)
    qc.add_M(2)
    qc.add_M(3)
else:
    print("Failed to make a circuit.")
    
# Run the circuit.
results = qc.run_circuit_get_results(shots=1024, seed=12345)
results['data']['simulateResult'][0]

{'startTime': '2019-05-07 05:00:22',
 'finishTime': '2019-05-07 05:00:22',
 'process': None,
 'data': {'111': '1.0'},
 'id': 10989,
 'seed': 12345,
 'shots': 1024,
 'measureQubits': [2, 1, 0]}

Let's compare this simulator with the QISKIT simulator we used in lab.

In [5]:
# Let's import the qiskit simulator and compare.
from qiskit import (
    QuantumCircuit, QuantumRegister, ClassicalRegister, Aer,
    execute
)
from qiskit.tools.visualization import plot_histogram

assert sys.version_info >= (3,5), "Only Python 3.5 or greater supported."

# import state tomography functions
from qiskit.tools.visualization import plot_histogram, plot_state

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

![hyperlink](https://i.ibb.co/1ngLJ8T/Capture.png)

In [30]:
# Code from QISKIT
# initalize registers
q, c = QuantumRegister(1), ClassicalRegister(1)

# initialize circuit
qc = QuantumCircuit(q, c)

# Add gate sequences
qc.h(q[0])
qc.s(q[0])
qc.t(q[0])
qc.h(q[0])
qc.measure(q[0], c[0])

# Run the circuit
backend_sim = Aer.get_backend('qasm_simulator')
results = execute(qc, backend_sim).result()
res = results.get_counts(qc)
total = res['0'] + res['1']
res['0'] = res['0'] / total
res['1'] = res['1'] / total
# res

In [25]:
# Code using our python interface
# initialize circuit
qc = QuantumCircuitToffoli('Single Qubit', cookies)
qc.new_circuit()

# Add gate sequences
qc.add_H(1)
qc.add_S(1)
qc.add_T(1)
qc.add_H(1)
qc.add_M(1)
    
# Run the circuit.
results = qc.run_circuit_get_results(shots=1024, seed=12345)
# results['data']['simulateResult'][0]['data']

![hyperlink](https://www.dropbox.com/s/iq9dmai17kyow5x/Screenshot%202018-03-21%2009.44.58.png?raw=1)

In [24]:
# initalize registers
q, c = QuantumRegister(2), ClassicalRegister(2)

# initialize circuit
qc = QuantumCircuit(q, c)

# Add gate sequences
qc.x(q[0])
qc.cx(q[0], q[1])
qc.measure(q[0], c[0])
qc.measure(q[1], c[1])

# run circuit
backend_sim = Aer.get_backend('qasm_simulator')
results = execute(qc, backend_sim).result()
res = results.get_counts(qc)
total = res['11']
res['11'] = res['11'] / total
# res

In [10]:
# Code using our python interface
# initialize circuit
qc = QuantumCircuitToffoli('Two Qubit', cookies)
qc.new_circuit()

# Add gate sequences
qc.add_X(1)
qc.add_CNOT(1, 2)
qc.add_M(1)
qc.add_M(2)
    
# Run the circuit.
results = qc.run_circuit_get_results(shots=1024, seed=12345)
# results['data']['simulateResult'][0]['data']